<div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 1em;">
    <div style="background: blue;"></div>
</div>

## Setup

We'll use the mpg dataset and predict whether a car has automatic or manual transmission.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydataset

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
mpg = pydataset.data('mpg')
mpg.trans = np.where(mpg.trans.str.startswith('auto'), 'auto', 'manual')

In [ ]:
X, y = mpg[['displ', 'cyl', 'cty', 'hwy']], mpg.trans

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

## `.cross_val_score`

In [ ]:
clf = DecisionTreeClassifier(max_depth=4)
cross_val_score(clf, X_train, y_train, cv=5).mean()

In [ ]:
results = []
for depth in range(1, 21):
    clf = DecisionTreeClassifier(max_depth=depth)
    result = {'depth': depth}
    result['score'] = cross_val_score(clf, X_train, y_train, cv=5).mean()
    results.append(result)
        
pd.DataFrame(results).set_index('depth').score.plot(xticks=range(1, 21))

## Grid Search

In [ ]:
clf = KNeighborsClassifier()
grid = GridSearchCV(clf, {'n_neighbors': range(1, 21)}, cv=5)
grid.fit(X_train, y_train)

In [ ]:
grid.cv_results_['mean_test_score']

In [ ]:
grid.cv_results_['params']

In [ ]:
grid.best_score_, grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
clf = DecisionTreeClassifier()
grid = GridSearchCV(clf, {'max_depth': range(1, 21), 'min_samples_leaf': range(1, 11)}, cv=10)
grid.fit(X_train, y_train)

In [ ]:
for params, score in zip(grid.cv_results_['params'], grid.cv_results_['mean_test_score']):
    params['score'] = score

In [ ]:
pd.DataFrame(grid.cv_results_['params']).sort_values(by='score').tail(10)

## Custom Scoring

In [ ]:
from sklearn.metrics import make_scorer, precision_score

clf = KNeighborsClassifier(n_neighbors=5)
cross_val_score(clf, X_train, y_train, scoring=make_scorer(precision_score, pos_label='manual'))